In [1]:
import numpy as np
from PIL import Image
import pandas as pd
%config Completer.use_jedi = False

In [5]:
image_names = !ls ../raw_data/images_test_rev1/
df_image_names = pd.DataFrame(image_names)
df_image_names = df_image_names.rename(columns={0:'image'})
df_training_solutions_rev1 = pd.read_csv('../raw_data/central_pixel_benchmark.csv')

In [6]:
df_training_solutions_rev1['image'] = df_image_names['image']
df_X_y = df_training_solutions_rev1.drop(columns=['GalaxyID'])
df_X_y.head(2)

,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,Class5.1,...,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6,image
0,0.468481,0.506185,0.025334,0.099844,0.406341,0.095167,0.311174,0.18508,0.221262,0.019015,...,0.082314,0.071253,0.031512,0.012281,0.079964,0.014346,0.006369,0.006649,0,100018.jpg
1,0.468481,0.506185,0.025334,0.099844,0.406341,0.095167,0.311174,0.18508,0.221262,0.019015,...,0.082314,0.071253,0.031512,0.012281,0.079964,0.014346,0.006369,0.006649,0,100037.jpg


In [10]:
import tensorflow as tf

def load_image(image):
    folder = f'../raw_data/images_test_rev1/{image}'
    img = Image.open(folder)
    img_array = np.array(img)
    return np.resize(img_array, (224,224,3))
    
def load_images(df):
    img_list = []
    for _, row in df.iterrows():
        img = load_image(row["image"])
        img_list.append(img) 
    return np.stack(img_list)

class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, df, batch_size=64, shuffle=True):
        self.batch_size = batch_size
        self.df = df
        self.indices = self.df.index.tolist()
        self.shuffle = shuffle
        self.on_epoch_end()
        

    def __len__(self):
        return len(self.indices) // self.batch_size
    
    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)
    
    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        X, y = self._get_data(batch)
        return X, y
    
    def _get_data(self, batch):
        df_batch = self.df.query("index in @batch") 
        X = self.__get_X_image(df_batch)
        y = self.__get_Y(df_batch)
        return X, y
    
    def __get_X_image(self, df):
        X = load_images(df)
        return X
            
    def __get_Y(self, df):
        return np.array(df.drop(columns=["image"]))

In [11]:
datagenerator = DataGenerator(df=df_X_y, shuffle=False)

In [12]:
datagenerator.__getitem__(0)[0].shape

(64, 224, 224, 3)